In [ ]:
#### Prepare
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display,clear_output

from highfret import aligner

fn_data = None

## initial guess to file
default = None
fns = os.listdir('./')
for fn in fns:
	if fn.endswith('.tif'):
		default = fn
		break

## build some widgets
text_filename = widgets.Textarea(value=default,placeholder='Enter file to align',description="File name",layout=widgets.Layout(width='80%',height='24pt'), style={'description_width':'initial'})
button_prepare = widgets.Button(description="Prepare Data")
int_start = widgets.IntText(value=0,description='First Frame')
int_end = widgets.IntText(value=0,description='Last Frame ')
dropdown_method = widgets.Dropdown(value='mean',options=['mean','acf','Frame 0'],ensure_option=True,description='Method:')
dropdown_split = widgets.Dropdown(value='L/R',options=['L/R','T/B'],ensure_option=True,description='Split:')
label_prepare = widgets.Label('')
vbox_prepare = widgets.VBox([text_filename,button_prepare,int_start,int_end,dropdown_method,dropdown_split,label_prepare])

def show_prep_ui():
	display(vbox_prepare)

def click_prepare(b):
	global fn_data
	filename = re.sub(r'\s+', '', text_filename.value)
	clear_output()
	show_prep_ui()
	if os.path.exists(filename):
		label_prepare.value = 'Found: %s'%(filename)
		meth = dropdown_method.value
		if meth == 'Frame 0': ## for now....
			meth = 0
		split = dropdown_split.value 
		first = int_start.value
		last = int_end.value
		aligner.prepare_data(filename,meth,split,first,last) ## Note: this makes the results directory, if necessary
		fn_data = filename
	else:
		label_prepare.value = 'Failure: File does not exist !!!! %s'%(filename)
		fn_data = None

button_prepare.on_click(click_prepare)
show_prep_ui()

In [ ]:
#### Align
if fn_data is None:
	raise Exception('Please prepare some data first')

results_index = None
results_names = []
results_thetas = []

out_dir = aligner.get_out_dir(fn_data)

#### Populate the results 
def get_index(s):
	return int(s.split('_')[0])
def save_theta(name,theta):
	global out_dir
	fn_out = os.path.join(out_dir,name+'.theta.npy')
	np.save(fn_out,theta)
	
fnthetas = [fn for fn in os.listdir(out_dir) if fn.endswith('.theta.npy')]
if len(fnthetas) == 0:
	results_names.append('%04d_blank_order1_bin1'%(0))
	results_thetas.append(aligner.initialize_theta(fn_data,False,None,False))
	save_theta(results_names[0],results_thetas[0])
	results_index = 0
else:
	indexes = [get_index(fn) for fn in fnthetas]
	order = np.argsort(indexes)
	for i in order:
		results_thetas.append(np.load(os.path.join(out_dir,fnthetas[i])))
		results_names.append(os.path.split(fnthetas[i])[1][:-10])
	results_index = len(results_names)-1

#### Make widgets
text_thetafilename = widgets.Textarea(value=None,placeholder='Enter Theta File Name',description="File Name",layout=widgets.Layout(width='80%',height='24pt'), style={'description_width':'initial'})
button_loadtheta = widgets.Button(description="Load Theta",layout=widgets.Layout(width='2in',height='0.25in'))
button_blanktheta = widgets.Button(description="Blank Theta",layout=widgets.Layout(width='2in',height='0.25in'))
button_fouriertheta = widgets.Button(description="Estimate Theta (Fourier)",layout=widgets.Layout(width='2in',height='0.25in'))

dropdown_results = widgets.Dropdown(value=results_names[results_index], options=results_names[::-1])

accordion_initializetheta = widgets.Accordion(children=[widgets.VBox([button_blanktheta,button_fouriertheta,text_thetafilename,button_loadtheta,]),], titles=('Initialize Theta',))

button_optimize = widgets.Button(description='Optimize')
dropdown_order = widgets.Dropdown(value=1, options=[1,2,3,4,5],description='Poly. Order')
dropdown_downscale = widgets.Dropdown(value=1, options=[1,2,4,8],description='Downscale')
range_minmaxiter = widgets.IntRangeSlider(value=[5, 10],min=0,max=10,step=1,description='Iteration Limits:',orientation='horizontal',readout_format='d',layout=widgets.Layout(width='80%',height='24pt'), style={'description_width':'initial'})


#### Hook up buttons and things
def get_last_index(thetalist):
	max_index = np.max([get_index(tli) for tli in thetalist])
	return max_index

def add_new_theta(new_name,new_theta):
	results_names.append(new_name)
	results_thetas.append(new_theta)
	results_index = len(results_names)-1
	save_theta(results_names[results_index],results_thetas[results_index])
	update_resultsdropdown()
	dropdown_results.value = new_name

def on_combo_box_change(change):
	global results_index
	if change['type'] == 'change' and change['name'] == 'value':
		results_index = results_names.index(change['new'])

def update_resultsdropdown():
	dropdown_results.options = results_names[::-1]

def click_loadtheta(b):
	global fn_data
	filename = re.sub(r'\s+', '', text_thetafilename.value)
	if os.path.exists(filename):
		new_theta = aligner.initialize_theta(fn_data,True,filename,False)
		order = aligner.alignment.coefficients_order(new_theta)
		new_name = '%04d_loaded_order%d_binX'%(get_last_index(results_names)+1,order)
		add_new_theta(new_name,new_theta)
	else:
		print('Failure: File does not exist !!!! %s'%(filename))
		return

def click_blanktheta(b):
	global fn_data
	new_name = '%04d_blank_order%d_bin%d'%(get_last_index(results_names)+1,1,1)
	new_theta = aligner.initialize_theta(fn_data,False,None,False)
	add_new_theta(new_name,new_theta)

def click_fouriertheta(b):
	new_name = '%04d_fourier_order%d_bin%d'%(get_last_index(results_names)+1,1,1)
	new_theta = aligner.initialize_theta(fn_data,False,None,True)
	add_new_theta(new_name,new_theta)

def show_opt_ui():
	#### Setup geometry
	vbox_align = widgets.VBox([accordion_initializetheta,dropdown_results,range_minmaxiter,dropdown_order,dropdown_downscale,button_optimize])
	display(vbox_align)

def click_optimize(b):
	clear_output()
	show_opt_ui()

	order = int(dropdown_order.value)
	downscale = float(dropdown_downscale.value)
	miniter,maxiter = range_minmaxiter.value
	old_theta = results_thetas[results_index]

	new_theta = aligner.optimize_data(fn_data,old_theta,float(downscale),int(order),True,maxiter,miniter)
	new_name = '%04d_optimize_order%d_bin%d'%(get_last_index(results_names)+1,order,downscale)
	add_new_theta(new_name,new_theta)

dropdown_results.observe(on_combo_box_change)
button_loadtheta.on_click(click_loadtheta)
button_blanktheta.on_click(click_blanktheta)
button_fouriertheta.on_click(click_fouriertheta)
button_optimize.on_click(click_optimize)

show_opt_ui()

In [ ]:
#### Plot
def plot():
	out_dir = aligner.get_out_dir(fn_data)
	fig,ax = aligner.render_images(fn_data,results_thetas[results_index])
	fig.set_figheight(8.)
	fig.set_figwidth(20.)
	return fig,ax
fig,ax = plot()
plt.savefig(os.path.join(out_dir,'render_%s.png'%(results_names[results_index])))

dropdown_zoom = widgets.Dropdown(value='Full', options=['Full','Center','TL','TR','BL','BR'],description='Plot Zoom')
def change_zoom(change):
	if change['type'] == 'change' and change['name'] == 'value':
		clear_output()
		display(widgets.VBox([dropdown_zoom]))
		
		fig,ax = plot()
		ny,nx,_ = ax[0].images[0].get_array().shape
		ll = np.min((nx,ny))//4

		if dropdown_zoom.value == 'Full':
			ax[0].set_xlim(0,nx)
			ax[0].set_ylim(ny,0)
		elif dropdown_zoom.value == 'Center':
			ax[0].set_xlim(nx//2-ll,nx//2+ll)
			ax[0].set_ylim(ny//2+ll,ny//2-ll)
		elif dropdown_zoom.value == 'TL':
			ax[0].set_xlim(0,ll)
			ax[0].set_ylim(ll,0)
		elif dropdown_zoom.value == 'TR':
			ax[0].set_xlim(nx-ll,nx-1)
			ax[0].set_ylim(ll,0)
		elif dropdown_zoom.value == 'BL':
			ax[0].set_xlim(0,ll)
			ax[0].set_ylim(ny-1,ny-ll)
		elif dropdown_zoom.value == 'BR':
			ax[0].set_xlim(nx-ll,nx-1)
			ax[0].set_ylim(ny-1,ny-ll)
		fig.tight_layout()
		plt.show()

dropdown_zoom.observe(change_zoom)
display(widgets.VBox([dropdown_zoom]))
plt.show()

